<a href="https://colab.research.google.com/github/JonasSteves/CashLog/blob/master/Masterthesis_V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pulp as pl

class ElectricEngineSupplyChain:
    def __init__(self):
        # Named suppliers, manufacturing plants, distribution centers, and customer regions
        self.suppliers = ['Housing Supplier', 'Shaft Supplier', 'Rotor Supplier', 'Stator Supplier']
        self.manufacturing_plants = ['East Coast Plant', 'West Coast Plant']
        self.distribution_centers = ['Central Distribution Center']
        self.customer_regions = ['North Region']  # Only one demand region
        self.production_levels = ['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High']
        self.time_periods = ['T1']  # Only one time period
        self.transportation_modes = ['Plane', 'Train', 'Ship']

        # Example fixed costs for suppliers
        self.fixed_supplier_costs = {'Housing Supplier': 100, 'Shaft Supplier': 200, 'Rotor Supplier': 150, 'Stator Supplier': 250}

        # Example manufacturing costs for each production level
        self.manufacturing_costs = {
            'Low': {'Coefficient': -0.12432, 'Intercept': 1317.94},
            'Medium-Low': {'Coefficient': -0.01848, 'Intercept': 788.73},
            'Medium': {'Coefficient': -0.00684, 'Intercept': 672.31},
            'Medium-High': {'Coefficient': -0.00268, 'Intercept': 568.49},
            'High': {'Coefficient': -0.00086, 'Intercept': 477.51}
        }

        # Example operating costs for distribution centers
        self.distribution_center_costs = {'Central Distribution Center': 500}

        # Example transportation costs (per unit)
        self.transportation_costs = {
            ('Housing Supplier', 'East Coast Plant', 'Plane'): 8, ('Housing Supplier', 'East Coast Plant', 'Train'): 6, ('Housing Supplier', 'East Coast Plant', 'Ship'): 3,
            ('Housing Supplier', 'West Coast Plant', 'Plane'): 8, ('Housing Supplier', 'West Coast Plant', 'Train'): 6, ('Housing Supplier', 'West Coast Plant', 'Ship'): 3,
            ('Shaft Supplier', 'East Coast Plant', 'Plane'): 8, ('Shaft Supplier', 'East Coast Plant', 'Train'): 6, ('Shaft Supplier', 'East Coast Plant', 'Ship'): 3,
            ('Shaft Supplier', 'West Coast Plant', 'Plane'): 8, ('Shaft Supplier', 'West Coast Plant', 'Train'): 6, ('Shaft Supplier', 'West Coast Plant', 'Ship'): 3,
            ('Rotor Supplier', 'East Coast Plant', 'Plane'): 8, ('Rotor Supplier', 'East Coast Plant', 'Train'): 6, ('Rotor Supplier', 'East Coast Plant', 'Ship'): 3,
            ('Rotor Supplier', 'West Coast Plant', 'Plane'): 8, ('Rotor Supplier', 'West Coast Plant', 'Train'): 6, ('Rotor Supplier', 'West Coast Plant', 'Ship'): 3,
            ('Stator Supplier', 'East Coast Plant', 'Plane'): 8, ('Stator Supplier', 'East Coast Plant', 'Train'): 6, ('Stator Supplier', 'East Coast Plant', 'Ship'): 3,
            ('Stator Supplier', 'West Coast Plant', 'Plane'): 8, ('Stator Supplier', 'West Coast Plant', 'Train'): 6, ('Stator Supplier', 'West Coast Plant', 'Ship'): 3,
            ('East Coast Plant', 'Central Distribution Center', 'Plane'): 8, ('East Coast Plant', 'Central Distribution Center', 'Train'): 6, ('East Coast Plant', 'Central Distribution Center', 'Ship'): 3,
            ('East Coast Plant', 'Southern Distribution Center', 'Plane'): 8, ('East Coast Plant', 'Southern Distribution Center', 'Train'): 6, ('East Coast Plant', 'Southern Distribution Center', 'Ship'): 3,
            ('West Coast Plant', 'Central Distribution Center', 'Plane'): 8, ('West Coast Plant', 'Central Distribution Center', 'Train'): 6, ('West Coast Plant', 'Central Distribution Center', 'Ship'): 3,
            ('West Coast Plant', 'Southern Distribution Center', 'Plane'): 8, ('West Coast Plant', 'Southern Distribution Center', 'Train'): 6, ('West Coast Plant', 'Southern Distribution Center', 'Ship'): 3,
            ('Central Distribution Center', 'North Region', 'Plane'): 8, ('Central Distribution Center', 'North Region', 'Train'): 6, ('Central Distribution Center', 'North Region', 'Ship'): 3,
            ('Central Distribution Center', 'South Region', 'Plane'): 8, ('Central Distribution Center', 'South Region', 'Train'): 6, ('Central Distribution Center', 'South Region', 'Ship'): 3,
            ('Southern Distribution Center', 'North Region', 'Plane'): 8, ('Southern Distribution Center', 'North Region', 'Train'): 6, ('Southern Distribution Center', 'North Region', 'Ship'): 3,
            ('Southern Distribution Center', 'South Region', 'Plane'): 8, ('Southern Distribution Center', 'South Region', 'Train'): 6, ('Southern Distribution Center', 'South Region', 'Ship'): 3,
        }

        # Example demand forecast
        self.demand_forecast = {
            ('North Region', 'T1'): 100000
        }

    def solve(self):
        prob = pl.LpProblem('ElectricEngineSupplyChain', pl.LpMinimize)

        # Decision variables
        quantity_shipped = pl.LpVariable.dicts("QuantityShipped",
                                               [(i, j, v, t) for i in self.suppliers + self.manufacturing_plants + self.distribution_centers
                                                for j in self.manufacturing_plants + self.distribution_centers + self.customer_regions
                                                if i != j  # Ensure no self-referencing
                                                for v in self.transportation_modes
                                                for t in self.time_periods],
                                               lowBound=0, cat='Continuous')

        production_level_operation = pl.LpVariable.dicts("ProductionLevelOperation",
                                                         [(m, p, t) for m in self.manufacturing_plants
                                                          for p in self.production_levels
                                                          for t in self.time_periods],
                                                         cat='Binary')

        facility_operation = pl.LpVariable.dicts("FacilityOperation",
                                                 self.suppliers + self.manufacturing_plants + self.distribution_centers,
                                                 cat='Binary')

        # Objective function components
        fixed_costs = pl.lpSum(self.fixed_supplier_costs[s] * facility_operation[s] for s in self.suppliers)
        distribution_costs = pl.lpSum(self.distribution_center_costs[d] * facility_operation[d] for d in self.distribution_centers)

        # Transportation costs term
        transportation_costs_term = pl.lpSum(self.transportation_costs[(i, j, v)] * quantity_shipped[(i, j, v, t)]
                                             for i in self.suppliers + self.manufacturing_plants + self.distribution_centers
                                             for j in self.manufacturing_plants + self.distribution_centers + self.customer_regions
                                             for v in self.transportation_modes
                                             for t in self.time_periods
                                             if i != j and (i, j, v) in self.transportation_costs)  # Ensure valid keys

        # Manufacturing costs term using demand
        manufacturing_costs_term = pl.lpSum(
            production_level_operation[(m, 'Medium-High', t)] *
            ((self.manufacturing_costs['Medium-High']['Coefficient'] * (self.demand_forecast[('North Region', 'T1')] / 2) + self.manufacturing_costs['Medium-High']['Intercept'])
             * (self.demand_forecast[('North Region', 'T1')] / 2))
            for m in self.manufacturing_plants
            for t in self.time_periods
        )

        prob += fixed_costs + distribution_costs + transportation_costs_term + manufacturing_costs_term

        # Constraints

        # Flow balance constraints for suppliers to manufacturing plants
        for supplier in self.suppliers:
            for t in self.time_periods:
                prob += pl.lpSum(quantity_shipped[(supplier, plant, v, t)] for plant in self.manufacturing_plants
                                 for v in self.transportation_modes
                                 if (supplier, plant, v, t) in quantity_shipped) >= 0

        # Flow balance constraints for manufacturing plants to distribution centers
        for plant in self.manufacturing_plants:
            for t in self.time_periods:
                prob += pl.lpSum(quantity_shipped[(plant, dc, v, t)] for dc in self.distribution_centers
                                 for v in self.transportation_modes
                                 if (plant, dc, v, t) in quantity_shipped) >= 0

        # Flow balance constraints for distribution centers to customer regions
        for dc in self.distribution_centers:
            for t in self.time_periods:
                prob += pl.lpSum(quantity_shipped[(dc, region, v, t)] for region in self.customer_regions
                                 for v in self.transportation_modes
                                 if (dc, region, v, t) in quantity_shipped) >= 0

        # Demand satisfaction constraints
        for c in self.customer_regions:
            for t in self.time_periods:
                prob += pl.lpSum(quantity_shipped[(d, c, v, t)] for d in self.distribution_centers
                                 for v in self.transportation_modes if (d, c, v, t) in quantity_shipped) >= self.demand_forecast[(c, t)]

        # Production level selection constraints
        for m in self.manufacturing_plants:
            for t in self.time_periods:
                prob += pl.lpSum(production_level_operation[(m, p, t)] for p in self.production_levels) == 1

        # Logical constraints for facility operations
        for i in self.suppliers + self.manufacturing_plants + self.distribution_centers:
            for j in self.manufacturing_plants + self.distribution_centers + self.customer_regions:
                if i != j:  # Ensure no self-referencing
                    for v in self.transportation_modes:
                        for t in self.time_periods:
                            if (i, j, v, t) in quantity_shipped:
                                prob += quantity_shipped[(i, j, v, t)] <= 1e6 * facility_operation[i]

        # Ensure no negative quantities are shipped
        for var in quantity_shipped.values():
            prob += var >= 0

        # Solve the model
        prob.solve()

        # Calculate the cost components
        total_fixed_costs = sum(self.fixed_supplier_costs[s] * facility_operation[s].varValue for s in self.suppliers)
        total_distribution_costs = sum(self.distribution_center_costs[d] * facility_operation[d].varValue for d in self.distribution_centers)
        total_transportation_costs = sum(self.transportation_costs[(i, j, v)] * quantity_shipped[(i, j, v, t)].varValue
                                         for i in self.suppliers + self.manufacturing_plants + self.distribution_centers
                                         for j in self.manufacturing_plants + self.distribution_centers + self.customer_regions
                                         for v in self.transportation_modes
                                         for t in self.time_periods
                                         if i != j and (i, j, v) in self.transportation_costs)
        total_manufacturing_costs = sum(
            production_level_operation[(m, 'Medium-High', t)].varValue *
            ((self.manufacturing_costs['Medium-High']['Coefficient'] * (self.demand_forecast[('North Region', 'T1')] / 2) + self.manufacturing_costs['Medium-High']['Intercept'])
             * (self.demand_forecast[('North Region', 'T1')] / 2))
            for m in self.manufacturing_plants
            for t in self.time_periods
        )

        # Store results for visualization
        self.results = {
            'total_cost': pl.value(prob.objective),
            'fixed_costs': total_fixed_costs,
            'distribution_costs': total_distribution_costs,
            'transportation_costs': total_transportation_costs,
            'manufacturing_costs': total_manufacturing_costs,
            'shipment_quantities': {var.name: var.varValue for var in prob.variables() if var.varValue > 0 and "QuantityShipped" in var.name}
        }

        # Print the results once
        print(f"Total cost: {self.results['total_cost']}")
        print(f"  Fixed costs: {self.results['fixed_costs']}")
        print(f"  Distribution center costs: {self.results['distribution_costs']}")
        print(f"  Transportation costs: {self.results['transportation_costs']}")
        print(f"  Manufacturing costs: {self.results['manufacturing_costs']}")

        print("Shipment Quantities:")
        for var_name, var_value in self.results['shipment_quantities'].items():
            print(f"{var_name}: {var_value}")

        # Additional print statements for production levels and quantities
        print("Production levels and quantities per plant:")
        for m in self.manufacturing_plants:
            for t in self.time_periods:
                for p in self.production_levels:
                    if production_level_operation[(m, p, t)].varValue > 0:
                        production_quantity = sum(quantity_shipped[(m, j, v, t)].varValue
                                                  for j in self.distribution_centers + self.customer_regions
                                                  for v in self.transportation_modes
                                                  if (m, j, v, t) in quantity_shipped)
                        print(f"Plant: {m}, Time Period: {t}, Production Level: {p}, Production Quantity: {production_quantity}")

# Instantiate the class
supply_chain = ElectricEngineSupplyChain()

# Solve the optimization problem
supply_chain.solve()


Total cost: 300500.0
  Fixed costs: 0.0
  Distribution center costs: 500.0
  Transportation costs: 300000.0
  Manufacturing costs: 0.0
Shipment Quantities:
QuantityShipped_('Central_Distribution_Center',_'North_Region',_'Ship',_'T1'): 100000.0
Production levels and quantities per plant:
Plant: East Coast Plant, Time Period: T1, Production Level: High, Production Quantity: 0.0
Plant: West Coast Plant, Time Period: T1, Production Level: High, Production Quantity: 0.0
